In [28]:
from functools import reduce

WP0_num,WP0_mean,WP0_stddev,WP1_num,WP1_mean,WP1_stddev = 1000,4.000000,0.300000,1000,3.500000,0.200000
name = "%d,%.2f,%.2f,%d,%.2f,%.2f" % (WP0_num,WP0_mean,WP0_stddev,WP1_num,WP1_mean,WP1_stddev)

wrongAnswerCost = 800
NUMBER_OF_REPETITIONS = 10
NUMBER_OF_ITEMS = 1000

RELATIVEPATH = "Experiments/old-"

fcU = open(RELATIVEPATH + "%s/%d/Unstarred/costs" % (name,wrongAnswerCost),'r')
faU = open(RELATIVEPATH + "%s/%d/Unstarred/accuracies" % (name,wrongAnswerCost),'r')

averageCostsUnstarred = [0.0]*10
averageAccuraciesUnstarred = [0.0]*10
for lineNo in range(NUMBER_OF_REPETITIONS):
    pass
    #averageAccuraciesUnstarred = list(map(lambda x,y:x+y,[float(x) for x in faU.readline().rstrip().split(",")],averageAccuraciesUnstarred))
    #averageCostsUnstarred = list(map(lambda x,y:x+y,[float(x) for x in fcU.readline().rstrip().split(",")],averageCostsUnstarred))
#averageCostsUnstarred = list(map(lambda x: (x/NUMBER_OF_REPETITIONS), averageCostsUnstarred))
#averageAccuraciesUnstarred = list(map(lambda x: (x/NUMBER_OF_REPETITIONS), averageAccuraciesUnstarred))
#overallAverageCostUnstarred = reduce(lambda x,y: x+y, averageCostsUnstarred)/10.0
#overallAverageAccuracyUnstarred = reduce(lambda x,y: x+y, averageAccuraciesUnstarred)/10.0




In [7]:
import algorithms_utils as alg_utils
import pandas as pd
import numpy as np

WP0_num,WP0_mean,WP0_stddev,WP1_num,WP1_mean,WP1_stddev = 1000,4.000000,0.300000,1000,3.500000,0.200000
name = "%d,%.2f,%.2f,%d,%.2f,%.2f" % (WP0_num,WP0_mean,WP0_stddev,WP1_num,WP1_mean,WP1_stddev)
RELATIVEPATH = "Experiments/"

def round_to_3(value):
    return round(value, 3)

NUMBER_OF_REPETITIONS = 6
NUMBER_OF_ITEMS = 1000
loss_ratio = 5
total_cost_mean = 0.
initial_line = 8

wrong_answer_costs = [10,
100,
300,
500,
700,
900,
1000,
10000]

for wrongAnswerCost in wrong_answer_costs:
    
    losses = []
    recalls = []
    precisions = []
    costs = []

    for iteration in range(NUMBER_OF_REPETITIONS):
        iteration_results_file = open(RELATIVEPATH + "%s/%d/Unstarred/%d" % (name,wrongAnswerCost,iteration),'r')
        gt_vals = []
        classification_vals = []
        iteration_cost = 0.
        for i, line in enumerate(iteration_results_file):
            if (initial_line - 1 < i < (initial_line + NUMBER_OF_ITEMS - 1)):
                item_id, y_val, gt_val, item_cost, item_difficulty, item_real_difficulty,  = line.rstrip().split(",")
                classification_vals.append(int(y_val))
                gt_vals.append(int(gt_val))
                iteration_cost += float(item_cost)
        # end for results file

        costs.append(iteration_cost / NUMBER_OF_ITEMS)    
        loss, recall, precision = alg_utils.Metrics.compute_metrics(classification_vals, gt_vals, loss_ratio)
        losses.append(loss)
        recalls.append(recall)
        precisions.append(precision)
    # end for iterations

    print(f"Results for cost: {wrongAnswerCost}:")
    print(f"Cost mean: {round_to_3(np.mean(costs))} - Std: {round_to_3(np.std(costs))}")
    print(f"Recall mean: {round_to_3(np.mean(recalls))} - Std: {round_to_3(np.std(recalls))}")
    print(f"Precision mean: {round_to_3(np.mean(precisions))} - Std: {round_to_3(np.std(precisions))}")
    print(f"Loss mean: {round_to_3(np.mean(losses))} - Std: {round_to_3(np.std(losses))}")
    print("  ---  ")
    

Results for cost: 10:
Cost mean: 0.999 - Std: 0.0
Recall mean: 0.75 - Std: 0.015
Precision mean: 0.961 - Std: 0.009
Loss mean: 1.148 - Std: 0.069
  ---  
Results for cost: 100:
Cost mean: 3.969 - Std: 0.044
Recall mean: 0.885 - Std: 0.009
Precision mean: 0.988 - Std: 0.003
Loss mean: 0.525 - Std: 0.038
  ---  
Results for cost: 300:
Cost mean: 5.799 - Std: 0.073
Recall mean: 0.93 - Std: 0.013
Precision mean: 0.993 - Std: 0.002
Loss mean: 0.319 - Std: 0.055
  ---  
Results for cost: 500:
Cost mean: 7.565 - Std: 0.294
Recall mean: 0.953 - Std: 0.008
Precision mean: 0.995 - Std: 0.003
Loss mean: 0.213 - Std: 0.038
  ---  
Results for cost: 700:
Cost mean: 8.912 - Std: 0.121
Recall mean: 0.963 - Std: 0.004
Precision mean: 0.997 - Std: 0.002
Loss mean: 0.168 - Std: 0.019
  ---  
Results for cost: 900:
Cost mean: 9.372 - Std: 0.08
Recall mean: 0.972 - Std: 0.007
Precision mean: 0.998 - Std: 0.002
Loss mean: 0.127 - Std: 0.032
  ---  
Results for cost: 1000:
Cost mean: 9.651 - Std: 0.095
Reca

In [13]:
Random()

NameError: name 'Random' is not defined